In [ ]:
# Import packages
import requests
from bs4 import BeautifulSoup
import math
import re
import time
import codecs
import pandas as pd
import urllib
import os

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [ ]:
# Define functions
def convert_list_to_dic(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

In [ ]:
# Get the HTML code
url = "https://tribunalsdecisions.service.gov.uk/utiac?&page=1"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())

In [ ]:
# Retrieve relevant variables
total_cases = int(str(re.findall(r'<b>(.*)</b>', str(soup.find_all("b")[1]), re.DOTALL))[2:7])
div_pages = int(str(re.findall(r'-(.*)</b>', str(soup.find_all("b")[0]), re.DOTALL))[6:8])
total_pages = math.ceil(total_cases/div_pages)

In [ ]:
total_pages

In [ ]:
# Save numbers for the future
with open('../data/total_old_cases.txt', 'w') as f:
    f.write(str(total_cases))
    
with open('../data/total_old_pages.txt', 'w') as f:
    f.write(str(total_pages))    

In [ ]:
# Retrieve **NEW** documents
with open('../data/total_old_cases.txt', 'r') as f:
    total_old_cases = f.readlines()
    
with open('../data/total_old_pages.txt', 'r') as f:
    total_old_pages = f.readlines()
    
    
with open('../data/corpus_archive_asylum_new.txt', 'w') as f:
    for i in range(0,total_pages):
        print(i)
        url = 'https://tribunalsdecisions.service.gov.uk/utiac?&page=' + str(i+1)
        req = requests.get(url, headers)
        soup = BeautifulSoup(req.content, 'html.parser')  
        table_cases = soup.find_all("table", class_='decisions-table')
        list_cases = re.findall(r'<a href="(.*)"></a>', str(table_cases))
        for j in range(0, len(list_cases)):
            id_case_list.append(list_cases[j])
            url_case = 'https://tribunalsdecisions.service.gov.uk/' + list_cases[j]
            req_case = requests.get(url_case, headers)
            soup_case = BeautifulSoup(req_case.content, 'html.parser')  
            url_case_pdf_aux = str(soup_case.find_all("a", class_='pdf-file'))
            url_case_pdf = re.findall(r'href="(.*)">Download', url_case_pdf_aux)
            url_case_pdf_list.append(url_case_pdf)

            # Get the metadata
            meta_data_case = [item.get_text() for item in soup_case.select("span")]
            # Clean text:
            meta_data_case = [e.replace('\xa0', '') for e in meta_data_case]
            meta_data_case = [e.replace('\n', '') for e in meta_data_case]
            # Append metadata into list of cases
            meta_data_case = convert_list_to_dic(meta_data_case)
            details_cases.append(meta_data_case)

            # Save content on a .txt
            try: 
                text_case_aux = soup_case.find("div", class_='decision-inner').get_text(strip=True)
                f.write(text_case_aux)
                f.write('\n')
                f.write('\n')
                f.write('\n')
            except:
                url_case_doc_aux = str(soup_case.find_all("a", class_='doc-file'))
                url_case_doc = re.findall(r'href="(.*)">Download', url_case_doc_aux)
                urllib.request.urlretrieve(url_case_doc[0], 'asylum_case_doc')
                #print("Error!")
                #print(url_case)
                up_boolean = False
                crown_boolean = True
                with codecs.open('asylum_case_doc', 'r', encoding="ISO-8859-1") as f2:
                     for num, line in enumerate(f2, 1):
                        if('Upper Tribunal' in line):
                            up_boolean = True
                        if('CROWN COPYRIGHT' in line):
                            crown_boolean = False
                            f.write(line)
                        if up_boolean and crown_boolean:
                            f.write(line)
        time.sleep(10)
        
        
os.remove('asylum_case_doc')       
flat_url_case_pdf_list = [item for sublist in url_case_pdf_list for item in sublist]
df_new = pd.DataFrame({'id': id_case_list, 'url':flat_url_case_pdf_list, 'details_cases':details_cases, 'downloaded':downloaded_cases})  
df_new.to_csv('../data/metadata_cases_new.csv')            

In [ ]:
# Retrieve **ALL** documents
id_case_list = []
url_case_pdf_list = []
details_cases = []
downloaded_cases = []

with open('../data/corpus_archive_asylum.txt', 'w') as f:
    for i in range(0,total_pages):
        print(i)
        url = 'https://tribunalsdecisions.service.gov.uk/utiac?&page=' + str(i+1)
        req = requests.get(url, headers)
        soup = BeautifulSoup(req.content, 'html.parser')  
        table_cases = soup.find_all("table", class_='decisions-table')
        list_cases = re.findall(r'<a href="(.*)"></a>', str(table_cases))
        for j in range(0, len(list_cases)):
            #print(list_cases[j])
            id_case_list.append(list_cases[j])
            url_case = 'https://tribunalsdecisions.service.gov.uk/' + list_cases[j]
            req_case = requests.get(url_case, headers)
            soup_case = BeautifulSoup(req_case.content, 'html.parser')  
            url_case_pdf_aux = str(soup_case.find_all("a", class_='pdf-file'))
            url_case_pdf = re.findall(r'href="(.*)">Download', url_case_pdf_aux)
            url_case_pdf_list.append(url_case_pdf)

            # Get the metadata
            meta_data_case = [item.get_text() for item in soup_case.select("span")]
            # Clean text:
            meta_data_case = [e.replace('\xa0', ' ') for e in meta_data_case]
            meta_data_case = [e.replace('\n', ' ') for e in meta_data_case]
            # Append metadata into list of cases
            meta_data_case = convert_list_to_dic(meta_data_case)
            details_cases.append(meta_data_case)

            # Save content on a .txt
            try: #PDF
                text_case_aux = soup_case.find('div', class_='decision-inner').get_text(strip=True)
                f.write('CASE ID: ' + list_cases[j])
                f.write(text_case_aux)
                f.write('\n')
                f.write('\n')
                f.write('\n')
                downloaded_cases.append('OK')
            except: #DOC
                try: 
                    url_case_doc_aux = str(soup_case.find_all('a', class_='doc-file'))
                    url_case_doc = re.findall(r'href="(.*)">Download', url_case_doc_aux)
                    urllib.request.urlretrieve(url_case_doc[0], 'asylum_case_doc')
                    up_boolean = False
                    crown_boolean = True
                    with codecs.open('asylum_case_doc', 'r', encoding="ISO-8859-1") as f2:
                         for num, line in enumerate(f2, 1):
                            if('Upper Tribunal' in line):
                                up_boolean = True
                            if('CROWN COPYRIGHT' in line):
                                crown_boolean = False
                                f.write(line)
                            if up_boolean and crown_boolean:
                                f.write(line)
                    downloaded_cases.append('OK')
                except:
                    downloaded_cases.append('FAILED')
                    print('Impossible to retrieve data on case: ' + url_case)
        time.sleep(10)
                  
            
os.remove('asylum_case_doc')       
flat_url_case_pdf_list = [item for sublist in url_case_pdf_list for item in sublist]
df = pd.DataFrame({'id': id_case_list, 'url':flat_url_case_pdf_list, 'details_cases':details_cases, 'downloaded':downloaded_cases})  
df.to_csv('../data/metadata_cases.csv')                                             